<div>

> **Note**
>
> Code chunks run Python commands unless it starts with `%%bash`, in
> which case, those chunks run shell commands.

</div>

In this tutorial we will continue the analysis of the integrated
dataset. We will use the scanpy enbedding to perform the clustering
using graph community detection algorithms.

Let's first load all necessary libraries and also the integrated dataset
from the previous step.

In [ ]:
import numpy as np
import pandas as pd
import scanpy as sc
import matplotlib.pyplot as plt
import warnings
import os
import urllib.request

warnings.simplefilter(action="ignore", category=Warning)

# verbosity: errors (0), warnings (1), info (2), hints (3)
sc.settings.verbosity = 3
sc.settings.set_figure_params(dpi=80)

In [ ]:
# download pre-computed data if missing or long compute
fetch_data = True

# url for source and intermediate data
path_data = "https://export.uppmax.uu.se/naiss2023-23-3/workshops/workshop-scrnaseq"

path_results = "data/covid/results"
if not os.path.exists(path_results):
    os.makedirs(path_results, exist_ok=True)

path_file = "data/covid/results/scanpy_covid_qc_dr_int.h5ad"
if fetch_data and not os.path.exists(path_file):
    urllib.request.urlretrieve(os.path.join(
        path_data, 'covid/results/scanpy_covid_qc_dr_int.h5ad'), path_file)

adata = sc.read_h5ad(path_file)
adata

## Graph clustering

The procedure of clustering on a Graph can be generalized as 3 main
steps:\
- Build a kNN graph from the data.\
- Prune spurious connections from kNN graph (optional step). This is a
SNN graph.\
- Find groups of cells that maximizes the connections within the group
compared other groups.

In Scanpy we do not build an SNN graph, instead the community detection
is done on the KNN graph which we construct using the command
`sc.pp.neighbors()`.

The main options to consider are:

-   **n_pcs** - the number of dimensions from the initial reduction to
    include when calculating distances between cells.
-   **n_neighbors** - the number of neighbors per cell to include in the
    KNN graph.

In this case, we will use the integrated data using Harmony. If you
recall, we stored the harmony reduction in `X_pca_harmony` in the
previous lab.

In [ ]:
sc.pp.neighbors(adata, n_neighbors=20, n_pcs=30, use_rep='X_pca_harmony')

# We will also set the default umap to the one created with harmony
# so that sc.pl.umap selects that embedding.
adata.obsm["X_umap"] = adata.obsm["X_umap_harmony"]

The modularity optimization algoritm in Scanpy is *Leiden*. Previously
ther was also *Louvain*, but since the Louvain algorithm is no longer
maintained, using Leiden is recommended by the Scanpy community.

### Leiden

In [ ]:
# default resolution is 1.0, but we will try a few different values.
sc.tl.leiden(adata, resolution = 0.4, key_added = "leiden_0.4")
sc.tl.leiden(adata, resolution = 0.6, key_added = "leiden_0.6")
sc.tl.leiden(adata, resolution = 1.0, key_added = "leiden_1.0")
sc.tl.leiden(adata, resolution = 1.4, key_added = "leiden_1.4")

Plot the clusters, as you can see, with increased resolution, we get
higher granularity in the clustering.

In [ ]:
sc.pl.umap(adata, color=['leiden_0.4', 'leiden_0.6', 'leiden_1.0','leiden_1.4'], legend_fontsize=8)

Once we have done clustering, the relationships between clusters can be
calculated as correlation in PCA space and we also visualize some of the
marker genes that we used in the Dim Reduction lab onto the clusters. If
we set `dendrogram=True` the clusters are ordered by the dendrogram in
the dotplot.

In [ ]:
sc.tl.dendrogram(adata, groupby = "leiden_0.6")
sc.pl.dendrogram(adata, groupby = "leiden_0.6")

genes  = ["CD3E", "CD4", "CD8A", "GNLY","NKG7", "MS4A1","FCGR3A","CD14","LYZ","CST3","MS4A7","FCGR1A"]
sc.pl.dotplot(adata, genes, groupby='leiden_0.6', dendrogram=True)

## K-means clustering

K-means is a generic clustering algorithm that has been used in many
application areas. In R, it can be applied via the `kmeans()` function.
Typically, it is applied to a reduced dimension representation of the
expression data (most often PCA, because of the interpretability of the
low-dimensional distances). We need to define the number of clusters in
advance. Since the results depend on the initialization of the cluster
centers, it is typically recommended to run K-means with multiple
starting configurations (via the `nstart` argument).

In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score

# extract pca coordinates
X_pca = adata.obsm['X_pca_harmony'] 

# kmeans with k=5
kmeans = KMeans(n_clusters=5, random_state=0).fit(X_pca) 
adata.obs['kmeans5'] = kmeans.labels_.astype(str)

# kmeans with k=10
kmeans = KMeans(n_clusters=10, random_state=0).fit(X_pca) 
adata.obs['kmeans10'] = kmeans.labels_.astype(str)

# kmeans with k=15
kmeans = KMeans(n_clusters=15, random_state=0).fit(X_pca)
adata.obs['kmeans15'] = kmeans.labels_.astype(str)

sc.pl.umap(adata, color=['kmeans5', 'kmeans10', 'kmeans15'])

adata.obsm

## Hierarchical clustering

Hierarchical clustering is another generic form of clustering that can
be applied also to scRNA-seq data. As K-means, it is typically applied
to a reduced dimension representation of the data. Hierarchical
clustering returns an entire hierarchy of partitionings (a dendrogram)
that can be cut at different levels. Hierarchical clustering is done in
these steps:

1.  Define the distances between samples. The most common are Euclidean
    distance (a.k.a. straight line between two points) or correlation
    coefficients.
2.  Define a measure of distances between clusters, called *linkage*
    criteria. It can for example be average distances between clusters.
    Commonly used methods are `single`, `complete`, `average`, `median`,
    `centroid` and `ward`.
3.  Define the dendrogram among all samples using **Bottom-up** or
    **Top-down** approach. **Bottom-up** is where samples start with
    their own cluster which end up merged pair-by-pair until only one
    cluster is left. **Top-down** is where samples start all in the same
    cluster that end up being split by 2 until each sample has its own
    cluster.

As you might have realized, correlation is not a method implemented in
the `dist()` function. However, we can create our own distances and
transform them to a distance object. We can first compute sample
correlations using the `cor` function.\
As you already know, correlation range from -1 to 1, where 1 indicates
that two samples are closest, -1 indicates that two samples are the
furthest and 0 is somewhat in between. This, however, creates a problem
in defining distances because a distance of 0 indicates that two samples
are closest, 1 indicates that two samples are the furthest and distance
of -1 is not meaningful. We thus need to transform the correlations to a
positive scale (a.k.a. **adjacency**):\
$$adj = \frac{1- cor}{2}$$\
Once we transformed the correlations to a 0-1 scale, we can simply
convert it to a distance object using `as.dist()` function. The
transformation does not need to have a maximum of 1, but it is more
intuitive to have it at 1, rather than at any other number.

The function `AgglomerativeClustering` has the option of running with
disntance metrics "euclidean", "l1", "l2", "manhattan", "cosine", or
"precomputed". However, with ward linkage only euklidean distances
works. Here we will try out euclidean distance and ward linkage
calculated in PCA space.

In [ ]:
from sklearn.cluster import AgglomerativeClustering

cluster = AgglomerativeClustering(n_clusters=5, linkage='ward')
adata.obs['hclust_5'] = cluster.fit_predict(X_pca).astype(str)

cluster = AgglomerativeClustering(n_clusters=10, linkage='ward')
adata.obs['hclust_10'] = cluster.fit_predict(X_pca).astype(str)

cluster = AgglomerativeClustering(n_clusters=15, linkage='ward')
adata.obs['hclust_15'] = cluster.fit_predict(X_pca).astype(str)

sc.pl.umap(adata, color=['hclust_5', 'hclust_10', 'hclust_15'])

Finally, lets save the clustered data for further analysis.

In [ ]:
adata.write_h5ad('./data/covid/results/scanpy_covid_qc_dr_int_cl.h5ad')

## Distribution of clusters

Now, we can select one of our clustering methods and compare the
proportion of samples across the clusters.

Select the "leiden_0.6" and plot proportion of samples per cluster and
also proportion covid vs ctrl.

Plot proportion of cells from each condition per cluster.

In [ ]:
tmp = pd.crosstab(adata.obs['leiden_0.6'],adata.obs['type'], normalize='index')
tmp.plot.bar(stacked=True).legend(bbox_to_anchor=(1.4, 1), loc='upper right')

tmp = pd.crosstab(adata.obs['leiden_0.6'],adata.obs['sample'], normalize='index')
tmp.plot.bar(stacked=True).legend(bbox_to_anchor=(1.4, 1),loc='upper right')

In this case we have quite good representation of each sample in each
cluster. But there are clearly some biases with more cells from one
sample in some clusters and also more covid cells in some of the
clusters.

We can also plot it in the other direction, the proportion of each
cluster per sample.

In [ ]:
tmp = pd.crosstab(adata.obs['sample'],adata.obs['leiden_0.6'], normalize='index')
tmp.plot.bar(stacked=True).legend(bbox_to_anchor=(1.4, 1), loc='upper right')

<div>

> **Discuss**
>
> By now you should know how to plot different features onto your data.
> Take the QC metrics that were calculated in the first exercise, that
> should be stored in your data object, and plot it as violin plots per
> cluster using the clustering method of your choice. For example, plot
> number of UMIS, detected genes, percent mitochondrial reads. Then,
> check carefully if there is any bias in how your data is separated by
> quality metrics. Could it be explained biologically, or could there be
> a technical bias there?

</div>

In [ ]:
sc.pl.violin(adata, ['n_genes_by_counts', 'total_counts', 'pct_counts_mt'], jitter=0.4, groupby = 'leiden_0.6', rotation= 45)

Some clusters that are clearly defined by higher number of genes and
counts. These are either doublets or a larger celltype. And some
clusters with low values on these metrics that are either low quality
cells or a smaller celltype. You will have to explore these clusters in
more detail to judge what you believe them to be.

## Subclustering of T and NK-cells

It is common that the subtypes of cells within a cluster is not so well
separated when you have a heterogeneous dataset. In such a case it could
be a good idea to run subclustering of individual celltypes. The main
reason for subclustering is that the variable genes and the first
principal components in the full analysis are mainly driven by
differences between celltypes, while with subclustering we may detect
smaller differences between subtypes within celltypes.

So first, lets find out where our T-cell and NK-cell clusters are. We
know that T-cells express CD3E, and the main subtypes are CD4 and CD8,
while NK-cells express GNLY.

In [ ]:
# check with the lowest resolution
fig, axs = plt.subplots(2, 3, figsize=(10,8),constrained_layout=True)
sc.pl.umap(adata, color="leiden_0.4", ax=axs[0,0], show=False, legend_loc = "on data")
sc.pl.umap(adata, color="CD3E", ax=axs[0,1], show=False)
sc.pl.umap(adata, color="CD4", ax=axs[0,2], show=False)
sc.pl.umap(adata, color="CD8A", ax=axs[1,0], show=False)
sc.pl.umap(adata, color="GNLY", ax=axs[1,1], show=False)

We can clearly see what clusters are T-cell clusters, so lets subset the
data for those cells

In [ ]:
tcells = adata[adata.obs["leiden_0.4"].isin(['1','2','4']),:]
tcells.obs["sample"].value_counts()

Ideally we should rerun all steps of integration with that subset of
cells instead of just taking the joint embedding. If you have too few
cells per sample in the celltype that you want to cluster it may not be
possible. We will start with selecting a new set of genes that better
reflecs the variability within this celltype

In [ ]:
sc.pp.highly_variable_genes(tcells, min_mean=0.0125, max_mean=3, min_disp=0.5)


print("Full data:" , sum(adata.var.highly_variable ))
print("Tcells:" , sum(tcells.var.highly_variable))
print("Intersection:" , sum(tcells.var.highly_variable & adata.var.highly_variable))

We clearly have a very different geneset now, so hopefully it should
better capture the variability within T-cells.

Now we have to run the full pipeline with scaling, pca, integration and
clustering on this subset of cells, using the new set of variable genes

In [ ]:
import scanpy.external as sce 
import harmonypy as hm 

tcells.raw = tcells
tcells = tcells[:, tcells.var.highly_variable]
sc.pp.regress_out(tcells, ['total_counts', 'pct_counts_mt'])
sc.pp.scale(tcells, max_value=10)
sc.tl.pca(tcells, svd_solver='arpack')
sce.pp.harmony_integrate(tcells, 'sample')
sc.pp.neighbors(tcells, n_neighbors=10, n_pcs=30, use_rep='X_pca_harmony')
sc.tl.leiden(tcells, resolution = 0.6, key_added = "tcells_0.6")
sc.tl.umap(tcells)

In [ ]:
fig, axs = plt.subplots(2, 3, figsize=(10,8),constrained_layout=True)
sc.pl.umap(tcells, color="sample", title="Tcell umap", ax=axs[0,0], show=False)
sc.pl.embedding(tcells, 'X_umap_harmony', color="sample", title="Full umap", ax=axs[1,0], show=False)
sc.pl.umap(tcells, color="leiden_0.6", title="Tcell umap, full clust", ax=axs[0,1], show=False)
sc.pl.embedding(tcells, 'X_umap_harmony', color="leiden_0.6", title="Full umap, full clust", ax=axs[1,1], show=False)
sc.pl.umap(tcells, color="tcells_0.6", title="Tcell umap, tcell clust", ax=axs[0,2], show=False)
sc.pl.embedding(tcells, 'X_umap_harmony', color="tcells_0.6", title="Full umap, tcell clust", ax=axs[1,2], show=False)

As you can see, we do have some new clusters that did not stand out
before. But in general the separation looks very similar.

We can plot the subtype genes again. If you try plotting the genes with
`use_raw=False` you will notice that some of the genes are not in the
`adata.X` matrix. Since they are no longer included in the variable
genes. So now we have to plot with `use_raw=True`.

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(10,8),constrained_layout=True)
sc.pl.umap(tcells, color="CD3E", ax=axs[0,0], show=False, use_raw=True)
sc.pl.umap(tcells, color="CD4", ax=axs[0,1], show=False, use_raw=True)
sc.pl.umap(tcells, color="CD8A", ax=axs[1,0], show=False, use_raw=True)
sc.pl.umap(tcells, color="GNLY", ax=axs[1,1], show=False, use_raw=True)

## Session info

```{=html}
<details>
```
```{=html}
<summary>
```
Click here
```{=html}
</summary>
```

In [ ]:
sc.logging.print_versions()

```{=html}
</details>
```